# Incoporation of heterologous genes in the iYO844 model for artemisinin production

_B. subtilis_ should be capable of producing farnesyl pyrophosphate (FPP), which is the end product of the MEP pathway. For production of artemisinin, 4 additional reactions are required.

In [6]:
# Loading of required packages
from cobra.io import read_sbml_model, write_sbml_model # To read the model and save new newly generated model
from cobra import Reaction, Metabolite # To creat new reactions and metabolites
from cameo.strain_design import pathway_prediction # To predict if there exist a pathway for a given metabolite

In [5]:
from cameo import models # HERE THE CAMEO CAN BE USED TO ACCESS THE MODELS FROM BIGG AND UNIVERSITY OF MINHO
models.bigg.iYO844 
# models.minho.iMM904 # MINHO NOT WORKING AS THEIR WEBPAGE IS DOWN

Name,iYO844
Memory address,0x07fc525ab3550
Number of metabolites,990
Number of reactions,1250
Number of groups,0
Objective expression,1.0*BIOMASS_BS_10 - 1.0*BIOMASS_BS_10_reverse_8788b
Compartments,"cytosol, extracellular space"


In [7]:
# Loading of the model
iYO844 = read_sbml_model("data/iYO844.xml")
iYO844_add = iYO844.copy() # Generating a copy of iY0844 to add the reactions to the model
iYO844

Name,iYO844
Memory address,0x07fc53f0473d0
Number of metabolites,990
Number of reactions,1250
Number of groups,0
Objective expression,1.0*BIOMASS_BS_10 - 1.0*BIOMASS_BS_10_reverse_8788b
Compartments,"cytosol, extracellular space"


First we make sure, that the MEP pathway is present in the current model iY40844 by searching the last two products of the pathway Geranyl diphosphate (GPP) and FPP.

In [8]:
iYO844.metabolites.get_by_id('grdp_c')

Metabolite identifier,grdp_c
Name,Geranyl diphosphate
Memory address,0x07fc53099c550
Formula,C10H17O7P2
Compartment,c
In 2 reaction(s),"DMATT, GRTT"


In [9]:
iYO844.metabolites.get_by_id('frdp_c')

Metabolite identifier,frdp_c
Name,Farnesyl diphosphate
Memory address,0x07fc530a6c910
Formula,C15H25O7P2
Compartment,c
In 3 reaction(s),"FRTT, GRTT, HEMEOS_1"


GRTT stands for geranyltranstransferase and both IspA: a geranyl pyrophosphate synthase + farnesyl pyrophosphate synthase and farnesyl pyrophosphate synthase (FPPS) is of this type of enzyme. 

In [10]:
# NOT WORKING ATM: From https://cameo.bio/07-predict-heterologous-pathways.html
# Trying to predict if there already exists a patwhay for the end product, which there should
#predictor = pathway_prediction.PathwayPredictor(iYO844)
#pathways = predictor.run(product='frdp_c', max_predictions=4)

### First reaction
In the article "High level production of amorphadiene using Bacillus subtilis as an optimized terpenoid cell factory", a FPPS from  _Saccharomyces cerevisiae_ was inserted to convert geranyl pyrophosphate (GPP) to farnesyl pyrophosphate (FPP) as the last step in the MEP pathway instead of using the native IspA as FPPS. \

GPP --> FPP
Catalysed by FPPS

In [41]:
reaction_1 = Reaction('FPPS') # farnesyl pyrophosphate synthase

# Constructing the reaction
reaction_1.add_metabolites({iYO844.metabolites.get_by_id('grdp_c'): -1,
                            iYO844.metabolites.get_by_id('frdp_c'): 1
                            })

print(reaction_1.build_reaction_string()) # Printing how the reaction looks
iYO844_add.add_reactions([reaction_1]) # Adding the reaction to the new iY0844 model

grdp_c --> frdp_c


In [49]:
iYO844_add.reactions.FPPS # Bounderies needs to be change?

Reaction identifier,FPPS
Name,
Memory address,0x07fde0c868850
Stoichiometry,grdp_c --> frdp_c Geranyl diphosphate --> Farnesyl diphosphate
GPR,
Lower bound,0.0
Upper bound,1000.0


### Second reaction
Artemisinin precusor: amorphadiene \
FPP --> amorphadiene / amorpha-4,11-diene \
Catalysed by amorphadiene synthase (ADS)

In [43]:
reaction_2 = Reaction('ADS') # amorphadiene synthase
amorphadiene = Metabolite(id='amorph_c', compartment='c') # Defining the new metabolite, as it does not exists

reaction_2.add_metabolites({iYO844.metabolites.get_by_id('frdp_c'): -1,
                            amorphadiene: 1
                            })
print(reaction_2.build_reaction_string())
iYO844_add.add_reactions([reaction_2])

frdp_c --> amorph_c


In [47]:
iYO844_add.reactions.ADS # Bounderies needs to be change?

Reaction identifier,ADS
Name,
Memory address,0x07fde0c868dd0
Stoichiometry,frdp_c --> amorph_c Farnesyl diphosphate -->
GPR,
Lower bound,0.0
Upper bound,1000.0


Testing to see if it can be produced

In [44]:
# Comments needs to be added to this
testing_product = iYO844_add.add_boundary(iYO844_add.metabolites.get_by_id('amorph_c'), type='demand')

with iYO844_add:
    iYO844_add.objective = testing_product
    sol = iYO844_add.optimize()
    print(sol.objective_value)

0.5191836734693878


### Third reaction
From "Production of the antimalarial drug precursor artemisinic acid in engineered yeast":

Oxidizing amorphadiene / amorpha-4,11-diene to artemisinic acid
Catalysed by CYP71AV1 a P450 enzyme (sesquiterpene lactone biosynthetic enzyme) from the plans _Artemisia annua_. 

The current model is saved as "iYO844_added.xml"

In [50]:
write_sbml_model(iYO844_add, 'data/iYO844_added.xml')